In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from typing import Iterable
import libs.padding as P

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils import data

dev = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(dev)
print(device)

cuda


In [15]:
bsz, channels, depth, height, width = 4, 3, 240, 240, 240
input_size = (bsz, channels, depth, height, width)

In [45]:
_kernel_size = (3, 3, 3)
_stride = (1, 2, 3)
_dilation = (1, 2, 3)
_padding = (1, 1, 1)

X = torch.rand(input_size)
print(X.shape)

pad = P.Pad3D(padding=_padding)
X = pad(X)
print(X.shape)

torch.Size([4, 3, 240, 240, 240])
torch.Size([4, 3, 242, 242, 242])


In [47]:
__correction = ((torch.Tensor(_dilation) * torch.Tensor(_stride))-torch.Tensor(_dilation)).type(torch.int)
print(__correction)

dstarts = torch.arange(0, depth-_kernel_size[0], _stride[0])
dends = torch.arange(_kernel_size[0]+(_stride[0]*_dilation[0] - __correction[0]), depth, _stride[0])
dstarts = dstarts[:dends.shape[0]]

hstarts = torch.arange(0, height-_kernel_size[1], _stride[1])
hends = torch.arange(_kernel_size[1]+(_stride[1]*_dilation[1] - __correction[1]), height, _stride[1])
hstarts = hstarts[:hends.shape[0]]

wstarts = torch.arange(0, width-_kernel_size[2], _stride[2])
wends = torch.arange(_kernel_size[2]+(_stride[2]*_dilation[2] - __correction[2]), width, _stride[2])
wstarts = wstarts[:wends.shape[0]]

print(dstarts[:5], hstarts[:5], wstarts[:5])
print(dends[:5], hends[:5], wends[:5])

tensor([0, 2, 6], dtype=torch.int32)
tensor([0, 1, 2, 3, 4]) tensor([0, 2, 4, 6, 8]) tensor([ 0,  3,  6,  9, 12])
tensor([4, 5, 6, 7, 8]) tensor([ 5,  7,  9, 11, 13]) tensor([ 6,  9, 12, 15, 18])
